# Complex TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os
import pandas as pd

from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [4]:
with open('../inclined/model.pickle') as w:
        model = pickle.load(w)

### Observation points and observed data

In [5]:
with open('data.pickle') as w:
        d = pickle.load(w)

In [6]:
data = pd.read_csv('inclined_model_data.txt', skipinitialspace=True, delim_whitespace=True)

In [7]:
dobs = data['res_data'].get_values()
xp = data['x'].get_values()
yp = data['y'].get_values()
zp = data['z'].get_values()
N = xp.size

### Parameters of the initial model

In [8]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 10.
int_max = 15.
intensity = np.linspace(int_min, int_max, 6)

# depth to the top, thickness and radius
z0_min = 0.
z0_max = 100.
z0 = np.linspace(z0_min, z0_max, 6)
dz = 800.
r = 700.
x0 = -200.
y0 = 0.

# main field
inc, dec = d['main_field']

In [9]:
z0

array([  0.,  20.,  40.,  60.,  80., 100.])

In [10]:
intensity

array([10., 11., 12., 13., 14., 15.])

### Outcropping parameters

In [11]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Limits

In [12]:
# limits for parameters in meters
rmin = 10.
rmax = 1200.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 4000.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Derivatives

In [13]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)

delta = np.array([deltax, deltay, deltar, deltaz])

### Regularization parameters

In [14]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-3  # adjacent radial distances within each prism
a2 = 1.0e-3   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-6    # vertically adjacent origins
a6 = 1.0e-7   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-5     # zero order Tikhonov on thickness of each prism

alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

### Folder to save the results

In [15]:
foldername = ''

In [16]:
itmax = 30
itmax_marq = 10
tol = 1.0e-3     # stop criterion

### Inversion

In [17]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        print 'inversion: %d  top: %d  intensity: %d' % (j*z0.size + k, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([m_est, phi_list, model_list, dobs - d_fit])

inversion: 0  top: 0  intensity: 10
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 1.40214e+04  fin obj.: 6.36449e+03
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 6.36449e+03  fin obj.: 3.29176e+03
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 3.29176e+03  fin obj.: 1.31297e+03
it:  3   it_marq:  0   lambda: 1e-02   init obj.: 1.31297e+03  fin obj.: 7.09747e+02
it:  4   it_marq:  0   lambda: 1e-03   init obj.: 7.09747e+02  fin obj.: 2.17274e+03
it:  4   it_marq:  1   lambda: 1e-02   init obj.: 7.09747e+02  fin obj.: 1.46975e+03
it:  4   it_marq:  2   lambda: 1e-01   init obj.: 7.09747e+02  fin obj.: 5.81155e+02
it:  5   it_marq:  0   lambda: 1e-02   init obj.: 5.81155e+02  fin obj.: 8.88119e+02
it:  5   it_marq:  1   lambda: 1e-01   init obj.: 5.81155e+02  fin obj.: 2.01246e+02
it:  6   it_marq:  0   lambda: 1e-02   init obj.: 2.01246e+02  fin obj.: 3.05319e+03
it:  6   it_marq:  1   lambda: 1e-01   init obj.: 2.01246e+02  fin obj.: 1.42614e+03
it:  6   it_marq:  2   lambda

# Results

In [18]:
# output of inversion
inversion = dict()

inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity
inversion['results'] = inversion_results

### Saving results

In [19]:
if foldername == '':
    mypath = 'l2-tfa-inversion/multiple-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = 'l2_tfa_inversion/multiple-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

In [20]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)